## 10. Clustream Algorithm

Imports

In [0]:
# necessary imports
from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from scipy import stats

In [0]:
# Loading the data!
np.random.seed(42)   # random seeding

# Loading the data
digits = load_digits()
data = scale(digits.data)
data =  np.asmatrix(data)
np.random.shuffle(data)


In [21]:
# Init number - initial set of points stored on disk to make q microclusters priorily before starting the online micro cluster process
InitNumber = 100

# The value of InitNumber is chosen to be as large as permitted by the computational complexity of a k-means 
# algorithm creating q clusters.
q = 20 # number of micro-clusters

# extracting the top InitNumber points to be stored on disk initially  
starter_dataset = data[0:InitNumber, :]
print('Length of the starter dataset (for initializing q clusters): ', starter_dataset.shape)

# assigning the rest of the data to stream_data that will be used in online mirco clustering process
stream_data = data[InitNumber:len(data), :]
print('SHape of the stream dataset: ', stream_data.shape)


Length of the starter dataset (for initializing q clusters):  (100, 64)
SHape of the stream dataset:  (1697, 64)


In [22]:
# applying k means on the first InitNumber points to initialize the q microclusters
kmeans = KMeans(init='k-means++', n_clusters=q).fit(starter_dataset) # cluster centers

starter_clusters = kmeans.cluster_centers_
print(starter_clusters.shape) # 64 is the feature size and 20 is the number of microcluster centroids as number of microclusters are 20.

(20, 64)


In [23]:
# For InitNumber of points (starter_data)
# micro-cluster tuple (CF2x, CF1x, CF2t, CF1t, n)   
micro_clusters = [None]*q

# for traversing through the initial data and assigning xi in that data to one of the 20 microclusters created before starting the online microclustering
for i in range(0,len(starter_dataset)):
    starter_point = np.array(starter_dataset[i])

    # get minimum euclidean distance cluster
    dist = [ np.linalg.norm(starter_point - cluster) for cluster in starter_clusters ]
    cluster_id = np.argmin(dist)
    
    # add to micro-cluster
    cluster_tuple = np.array([np.square(starter_point),starter_point,np.square(i), i, 1])
    
    # if there are no tuples yet in that microcluster then initialize with the first else add the tuple with the microcluster
    if (micro_clusters[cluster_id] == None): 
        micro_clusters[cluster_id] = cluster_tuple
    else:
        micro_clusters[cluster_id] = micro_clusters[cluster_id] + cluster_tuple

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


###Function CluStream: 
**Input**: \
**record** - input data point, \
**timestep** - time,  \
**micro_clusters** - q micro_clusters at the present moment,  \
**t** - maximal boundary factor (The maximum boundary of the micro-cluster is defined as a factor of t of the RMS deviation of the data points in the cluster from the centroid. We define this as the maximal boundary factor. ) \
**r** -      heuristic maximal boundary factor ( default value = 2, For a cluster which contains only 1 point, the maximum boundary is choosen to be r times the maximum boundary of the next closest cluster. ) \
**m** -      approximation detail for timestamp (m last data points of the micro-cluster) \
**delta** -  When the least relevance stamp of any micro-cluster is below a user-defined threshold delta (it can be eliminated and a new micro-cluster can be created with a unique id corresponding to the newly arrived data point )
      

In [0]:
def CluStream(record, timestep, micro_clusters, t = 2, r = 2, m = 10, delta = 10):   
    Xik = np.array(record)
    
    # get minimum euclidean distance cluster
    # centroid M = CF1x / n = cluster[1]/cluster[4]
    dist = [ np.linalg.norm(Xik - (cluster[1] / cluster[4])) for cluster in micro_clusters ]
    dist_sorted = np.argsort(dist)
    cluster = dist_sorted[0]
    
    i = 0
    while True:
        cluster_id = dist_sorted[i]
        n = micro_clusters[cluster_id][4]

        if n > 1:
            # RMS deviation
            squared_sum = np.square(micro_clusters[cluster_id][1])
            sum_of_squared = micro_clusters[cluster_id][0]
            RMSD = np.sqrt(np.abs(sum_of_squared - (squared_sum / n)))
            maximal_boundary = np.linalg.norm(RMSD) * t
            if i > 0:
                maximal_boundary *= r
            break
          
        # find next closest cluster
        i += 1
        
    # if data point falls within the maximum boundary of the micro-cluster
    if dist[cluster] <= maximal_boundary:
        micro_clusters[cluster] = micro_clusters[cluster] + np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])
        print ("add to cluster")
        
    # else create a new micro-cluster 
    else: 
        # determine if it is safe to delete any of the current micro-clusters as outlier
        mean_timestamp = [ (cluster[3] / cluster[4]) for cluster in micro_clusters ]
        standard_deviation_timestamp = [ np.sqrt((cluster[2] / cluster[4]) - np.square((cluster[3] / cluster[4]))) for cluster in micro_clusters ]
        
        Z = []
        for i in range(0, len(micro_clusters)):
            mc = m
            if mc > micro_clusters[i][4]:
                mc = micro_clusters[i][4]
                
            Z.append(m / (2 * micro_clusters[i][4]))   # changed
        
        Z = np.array(Z)

        # finding relvance_stamp and calculating the index of the minimum of the relevance_stamp cluster
        relevance_stamp = mean_timestamp + Z * standard_deviation_timestamp
        least_recent_cluster = np.argmin(relevance_stamp)
        print (relevance_stamp[least_recent_cluster])
        
        # eliminate old cluster and create a new micro-cluster (if relevance stamp is lesser than a specified value, delete that cluster to make space for the new cluster)
        if relevance_stamp[least_recent_cluster] < delta: 
            micro_clusters[least_recent_cluster] = np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])
            print ("eliminated cluster")

        # else merge the two micro-clusters which are closest to one another
        else: 
            # search for two closest clusters
            minA_id = -1
            minB_id = -1
            min_dist = float("inf")
            for a in range (0, len(micro_clusters)):
                for b in range (a + 1, len(micro_clusters)):
                    d = np.linalg.norm((micro_clusters[b][1] / micro_clusters[b][4]) - (micro_clusters[a][1] / micro_clusters[a][4]))
                    if d < min_dist:
                        minA_id = a
                        minB_id = b
                        min_dist = d
                        
            # merge them 
            micro_clusters[minA_id] = micro_clusters[minA_id] + micro_clusters[minB_id]
            
            #c create new cluster
            #c newId = length
            micro_clusters[minB_id] = np.array([np.square(Xik), Xik, np.square(timestep), timestep, 1])
            print("merged cluster")
        
    return micro_clusters

### Function process_snapshots  (Pyramidical Time Frame Implmentation)
**Input**: \
T - Timestep at the current moment

**Process** - See that T belongs to which of the orders and add T accordingly. 
        that order is full, delete/remove the least recent snapshot taken. Maintain all this in Pyramidical Time frame structure.

In [0]:
def process_snapshots(T):

  # computing log_{alpha}(T) and flooring to get the max order possible at T time units
  value = np.log(T)/np.log(alpha)
  max_order = np.floor(value)

  if(main_list.get(0) != None):
    print(T, '  :  ', main_list[0])

  # traversing through each order 0 to floor(log_{alpha}(T))
  for j in range(int(max_order) + 1):

    # checking if T is divisible by alpha^i
    den = np.power(alpha, j)
    if(T % den == 0):    # if T is divisible by alpha^i, then add the tuple in order j

      # add the tuple T in order j
      # if the order j is not empty or say if order j exists, then add T in the order j 
      if(main_list.get(j) != None):   # if order exists

        # if length of order j is full, then delete least recent time unit from the order j list
        if(len(main_list[j]) == max_length):

          # delete or remove the most recent decision
          least_recent = main_list[j][0]
          value1 = np.power(alpha, j+1)

          # if the least_recent time unit is not divisible by alpha^(j+1), then that thing is not there in the next order j+1 and hence
          # we can delete it
          if(least_recent % value1 != 0): 
            main_list[j] = main_list[j][1:]  # removing from the order j list
            del snap_shot[least_recent]    # deleting the micocluster stored at least_recent time unit index
            main_list[j] = main_list[j] + (T,) 

          # else if the least_recent time unit is divisible by alpha^(j+1), then just remove it from order j, 
          # but dont delete it from where microclusters are stored
          else: 
            main_list[j] = main_list[j][1:]   # only removing from the order j list, not from where snapshots of microclusters are stored
            main_list[j] = main_list[j] + (T,)
          
        # else means jth order is not full, then add the Time index T in the jth order list  
        else: # add simply 
          main_list[j] = main_list[j] + (T,)

      # else if order j does not exists, make jth order by positioning it with jth index, and add T in that
      else: 
        main_list[j] = (T,)

### Function find_microclusters
**Input**: \
**k** - User defined - upper level number of macroclusters to be formed from the microclusters present in the specified time horizon \
**h** - time horizon \
**tc** - current time \
\
**Process** - to find the microclusters between the time frame tc and tc-h' where h' is the approximation to h

In [0]:
def find_microclusters(k, h, tc):
  # FINDING THE LIST OF TIMES PRESENT IN THE PYRAMIDICAL TIME FRAME AT THE TIME tc WHICH ARE LESS THAN tc-h 
  time_list = []                    # list of all the time units present in the pyramidical time frame

  for key,value in main_list.items():    # iterating over the pyramidical time frame dictionary - main_list  (order: list_of_time_tuples)
    for j in range(len(main_list[key])):  
      if(main_list[key][j] in time_list):     # if the time unit value already exists, dont add in the time_list
        continue
      else:
        if(main_list[key][j] <= (tc-h)):       # if the time unit value does not exists, and if the time value is less or equal than (tc-h),  add this in the time_list list
          time_list.append(main_list[key][j])


  time_list.sort()    # sort the time_list 
  first_frame = tc   # tc - current time
  second_frame = time_list[-1]   # tc-h' - previous time

  print(second_frame)
  print(first_frame)

  # FINDING THE ID LIST OF ALL THE MICROCLUSTERS STORED AT THE TIME UNIT first_frame = tc
  idList = []
  for i in range(len(snap_shot[first_frame])):
      idList.append(i)

  length = len(idList)
  firstMicro = [None]*length
  secondMicro = [None]*length
  final_micro = [None]*length

  # FINDING THE CORRESPONDING IDS PRESENT IN SECOND TIME FRAME  - tc-h' and subtracting the microclusters of those ids from the two time units
  for i in range(len(idList)):
    firstMicro[i] = snap_shot[first_frame][i]
    secondMicro[i] = snap_shot[second_frame][i]
    final_micro[i] = firstMicro[i] - secondMicro[i]    # final microclusters between the time tc-h' and tc

  print('\n\nMicroclusters at tc: \n')
  print(firstMicro)
  print('\n-----------------------------------------------------------------------------------------------------------------------------------------------------\n')
  print('\n\nMicroclusters at tc-h: \n')
  print(secondMicro)
  print('\n---------------------------------------------------------------------------------------------------------------------------------------------------\n')
  print('\n\nMicroclusters between the horzon: \n')
  print(final_micro)

  return final_micro

###Function find_macro_clusters 
**Input** : \
final_micro - which are the q microclusters outputted from the avove function which lies between tc and tc-h' \
\

**Process** - First need to find k seed points for k means algorithm to b applied on the q microclustsers to form k macroclusters. Then to apply the k means algorithm by iterating over a few iterations that assigns these q microclustrs to the k macroclusters and then finally outputting the final k macroclusters

In [0]:
def getCenter(cluster):
  return cluster[1]/cluster[4]

def distance(c1, c2):
  return np.linalg.norm(c1-c2)

def find_macroclusters(final_micro):
  #  1st OBJECTIVE: choose the initial k seed points
  # number_list      =  length - q, number of points in each of the microclusters - to choose the top k clusters having greatest number of points to initialize the initial k seeds
  #                     for k means clustering
  #
  # k_clusters_index =  length - k, indexes of the top k clusters having maximum number of points
  #
  #
  noOfIterations = 10   # number of iterations for kmeans algorithm
  number_list = []      # list of number of points in each of the q microclusters
  for i in range(len(final_micro)):
    number_list.append(final_micro[i][4])

  number_list_sorted = np.argsort(number_list)  # Indexes --- sort the list according to the number of points in the q microclusters 
  number_list_sorted = number_list_sorted[::-1]  # reversing the index list to make it in decreasing order
  k_clusters_index = number_list_sorted[0:k]    # taking the top k index that corresponds to top k micoclusters having max number of points


  # Finding the k centers based on the k microclusters found having the maximum number of points
  centers = [None]*k    
  for i in range(k):                             
    cluster = final_micro[k_clusters_index[i]]   # taking the cluster at the ith position of the sorted list (decreasing order)
    centers[i] = cluster[1]/cluster[4]           # assigning the centroids to the centers ( centroid = (x1+x2+...+xn)/n  which means CF1x/n  === cluster[1]/cluster[4])


  print('\n\n\n')
  # FOUND K CENTERS TILL NOW TO FORM THE K MACROCLUSTERS FROM THE Q MICROCLUSTERS

  # 2nd OBJECTIVE: K means algorithm
  # centers - containing the intial k centers
  # final_micro - q microclusters to be converted to k macroclusters
  # 
  # number_list      =  length - q, number of points in each of the microclusters - to choose the top k clusters having greatest number of points to initialize the initial k seeds
  #                     for k means clustering
  #
  # k_clusters_index =  length - k, indexes of the top k clusters having maximum number of points
  #
  macro_clusters = dict()
  # K MEANS ALGORITHM
  # Iterations for k means on the q microclusters to form k macroclusters
  for i in range(k):
      macro_clusters[i] = []     # assigning list at k keys

  for iter in range(1, noOfIterations):

    for cluster in final_micro:
      minDistance = distance(getCenter(cluster), centers[0])   # assigning mindist to the distance between the cluster and first center
      closestCluster = 0
      for i in range(1, k):                                    # iterating over all the centers to see the min distance
        dist = distance(getCenter(cluster), centers[i])             
        if(dist<minDistance):
          closestCluster = i                                  # assigming the number of the closes cluster
          minDistance = dist                                  # setting the mindist of the cluster from that center
      macro_clusters[closestCluster].append(cluster)          # after calculating the closest cluster out of the k macroclusters, assign that microcluster to be a part of that macrocluster

    # Find new centers
    for key,values in macro_clusters.items():             # iterate over all the k macroclusters
      clusters_list = values                              
      center = 0
      for j in range(len(values)):
        center = center + (clusters_list[j][1]/clusters_list[j][4])   # calculate new center for each of the k macroclusters by adding the center values of the clusters stored at the (key)th macrocluster 
      if(len(values)!=0):
        centers[key] = center/len(values)                               # and dividing by the number of clusters present in that macrocluster to find the new center or say new centroid for this (key)th macrocluster 


  # 3rd OBJECTIVE: OUTPUT FINAL MACROCLUSTERS - ADD ALL THE CLUSTERS IN THE MACROCLUSTER FORMED AFTER THE K MEANS ALGORITHM
  #
  #
  # ADD ALL THE CLUSTERS IN THE MACROCLUSTER FORMED AFTER THE K MEANS ALGORITHM
  final_macrocluster = [None]*k                             # FINAL OUTPUT - k macroclusters
  for key,values in macro_clusters.items():                # iterate over all the k macroclusters
    cluster_sum = 0
    for j in range(len(values)):
      cluster_sum = cluster_sum + macro_clusters[key][j]    # summing the microclusters present at the (key)th macrocluster to form the final (key)th macrocluster
    final_macrocluster[key] = cluster_sum                   # assigning the sum to the (key)th place of final_macrocluster output 

  return final_macrocluster

##Main Function

####**outputs** :
Process of adding the tuple into cluster, \
q microclusters at time tc, \
q microclusters at time tc-h', \
q final microclusters between the time frames tc and tc-h',\
and Final k macroclusters (FINAL OUTPUT)



In [28]:
alpha = 2
l = 2
main_list =  dict()   # {order: time_index_tuple}
snap_shot = dict()   # {time_index: microcluser}
max_length = np.power(alpha, l) + 1  # maximum number of snapshots allowed for a particular order
prev_micro = micro_clusters

# define values by the user - for macroclustering
tc = 1690                        # current time at which the user gives in input h (time horizon) and k (number of macroclusters it wants)
h = 100                         # user defined value - Time horizon 
k = 5                             # user defined value - Number of macroclusters

# Starting the process of online microclustering
for i in range(0, len(stream_data)):
    micro_clusters = CluStream(stream_data[i], i+1, prev_micro)
    snap_shot[i+1] = micro_clusters.copy()
    prev_micro = micro_clusters
    process_snapshots(i+1)

    if(i == 1690):
      # call macroclustering process
      final_micro = find_microclusters(k, h, tc)
      macroclusters1 = find_macroclusters(final_micro)
      print("\n\n\n\n FINAL MICROCLUSTERS: \n\n")
      print(macroclusters1)


Streaming output truncated to the last 5000 lines.
add to cluster
201   :   (196, 197, 198, 199, 200)
add to cluster
202   :   (197, 198, 199, 200, 201)
add to cluster
203   :   (198, 199, 200, 201, 202)
add to cluster
204   :   (199, 200, 201, 202, 203)
add to cluster
205   :   (200, 201, 202, 203, 204)
add to cluster
206   :   (201, 202, 203, 204, 205)
add to cluster
207   :   (202, 203, 204, 205, 206)
add to cluster
208   :   (203, 204, 205, 206, 207)
add to cluster
209   :   (204, 205, 206, 207, 208)
add to cluster
210   :   (205, 206, 207, 208, 209)
add to cluster
211   :   (206, 207, 208, 209, 210)
add to cluster
212   :   (207, 208, 209, 210, 211)
add to cluster
213   :   (208, 209, 210, 211, 212)
add to cluster
214   :   (209, 210, 211, 212, 213)
add to cluster
215   :   (210, 211, 212, 213, 214)
add to cluster
216   :   (211, 212, 213, 214, 215)
add to cluster
217   :   (212, 213, 214, 215, 216)
add to cluster
218   :   (213, 214, 215, 216, 217)
add to cluster
219   :   (214, 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide
